In [1]:
#import pandas for file handling and reading
import pandas as pd

In [2]:
#import regular expression for data pre-processing
import re
from string import punctuation
from nltk.stem import SnowballStemmer

In [3]:
#read the file
df=pd.read_csv('train.csv')

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
#check data type for column for further manipulation
df['question1'].dtype

dtype('O')

In [6]:
#convert data type of column to string
df['question1']=df['question1'].astype("string")

In [7]:
df['question1'].dtype

StringDtype

In [8]:
#convert data type of column to string
df['question2']=df['question2'].astype("string")

In [9]:
#create a method for preprocessing data
def preprocess(text):
    ##text = text.replace('None','')
    text = text.lower()
    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text)
    text = re.sub("b\.g\.", " bg ", text)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text)
    text = re.sub("\(s\)", " ", text)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    return text

In [10]:
#Check missing values
df['question1'].isna().sum()

1

In [11]:
#replace missing value with other string
df['question1'].fillna('missing', inplace=True)

In [12]:
df['question1'].isna().sum()

0

In [13]:
#Check missing values
df['question2'].isna().sum()

2

In [14]:
#replace missing value with other string
df['question2'].fillna('missing', inplace=True)

In [15]:
df['question1'].isna().sum()

0

In [16]:
#pre-process both the columns

In [17]:
df['question1']=df['question1'].apply(preprocess)

In [18]:
df['question2']=df['question2'].apply(preprocess)

In [ ]:
#import nltk for tokenization

In [19]:
from nltk.tokenize import word_tokenize 

In [35]:
#create list of words  for question1 and question2 save it in a list
X_list = word_tokenize(df['question1'][1])  
Y_list = word_tokenize(df['question2'][1])

In [20]:
l1 =[];l2 =[] 

In [37]:
rvector = X_list + Y_list 
for w in rvector: 
    if w in X_list: l1.append(1) # create a vector 
    else: l1.append(0) 
    if w in Y_list: l2.append(1) 
    else: l2.append(0)

In [21]:
c=0

In [39]:
#check similarity score for first row
for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
cosine = c / float((sum(l1)*sum(l2))**0.5) 
print("similarity: ", cosine)

similarity:  0.7759402897989853


In [22]:
similar=[]

In [23]:
#apply similarity check for all the rows in dataframe
for i in range(0,len(df['question1'])):
        X_list = word_tokenize(df['question1'][i])  
        Y_list = word_tokenize(df['question2'][i])
        l1 =[];l2 =[] 
        rvector = X_list + Y_list 
        for w in rvector: 
            if w in X_list: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_list: l2.append(1) 
            else: l2.append(0)
        c=0
        for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        ##print("similarity: ", cosine)
        if cosine>0.80:
            result=1
        else:
            result=0
        #print(result)
        similar.append(result)

In [24]:
similar[1]


0

In [25]:
count=0

In [26]:
#check the accuracy for the rows correctly predicted
for i in range(0, len(df['question1'])):
    if(df['is_duplicate'][i]==similar[i]):
        count = count+1;
    else:
        pass
    
print(count)
    

265141


In [27]:
count

265141

In [28]:
correct_similarity=(count/len(df['question1']))*100

In [29]:
correct_similarity

65.58188429097925

In [90]:
print('Cosine Similarity was',correct_similarity,'% effective')

Cosine Similarity was 59.205520789532265  % effective
